# Problem 2 – Fractional Parts of Cube Roots  
**Author:** Michael Ferry  
**Date:** October 2025  

This problem focuses on recreating the constants listed on page 11 of the **Secure Hash Standard (FIPS PUB 180-4)**.  
These constants are the **first 32 bits of the fractional parts of the cube roots** of the first 64 prime numbers.

We’ll:
1. Write a function `primes(n)` to generate the first n primes.  
2. Use it to calculate cube roots for the first 64 primes.  
3. Extract the fractional part, scale it by 2³², and convert to 32-bit integers.  
4. Display the results in hexadecimal.  
5. Compare them to the official constants from FIPS 180-4 (Section 4.2.2).


In [6]:
import numpy as np
from math import pow, floor

print("Problem 2 imports loaded successfully.")


Problem 2 imports loaded successfully.


### Step 1 — Generate the first n primes

A small helper function is needed to produce the first 64 primes. This version
uses basic trial division, which is more than enough for the size of this task.
The function returns the primes in a NumPy array for easier calculations later.


In [7]:
def primes(n: int) -> np.ndarray:
    """
    Return the first n prime numbers.
    """
    if n <= 0:
        return np.array([], dtype=int)

    found = []
    candidate = 2

    while len(found) < n:
        is_prime = True

        for p in found:
            if p * p > candidate:
                break
            if candidate % p == 0:
                is_prime = False
                break

        if is_prime:
            found.append(candidate)

        candidate += 1

    return np.array(found, dtype=int)


# Quick check
primes(10)


array([ 2,  3,  5,  7, 11, 13, 17, 19, 23, 29])

### Step 2 — Cube roots of the first 64 primes

Now that the primes are ready, the next step is to calculate the cube roots of
all 64 of them. NumPy makes this straightforward, and these values will be used
to extract the fractional parts in the next stage.


In [8]:
# Get the first 64 primes
prime_list = primes(64)

# Cube roots (using float power)
cube_roots = prime_list ** (1/3)

# quick look at the first few cube roots
cube_roots[:10]


array([1.25992105, 1.44224957, 1.70997595, 1.91293118, 2.22398009,
       2.35133469, 2.57128159, 2.66840165, 2.84386698, 3.07231683])

### Step 3 — Fractional parts and 32-bit conversion

The cube roots include a whole number and a fractional part. SHA-256 only uses
the fractional part, so each value is reduced by taking (value - floor(value)).
That fractional part is then scaled by 2³² and stored as a 32-bit unsigned
integer.


In [9]:
# Only get the fractional part of each cube root
fractional_parts = cube_roots - np.floor(cube_roots)

# Converts to 32bit integers by scaling and casting
constants_32bit = (fractional_parts * (2**32)).astype(np.uint32)

# quick print of the first few constants
constants_32bit[:10]


array([1116352408, 1899447441, 3049323471, 3921009573,  961987163,
       1508970993, 2453635748, 2870763221, 3624381080,  310598401],
      dtype=uint32)

### Step 4 — Convert the 32-bit values to hexadecimal

The last step is to take the 32 bit integers and show them in hexadecimal form.
This is the format used in the SHA-256. Each value is padded so
it always appears as an 8 character hex number.


In [10]:
# Converts each 32 bit integer to an 8 char hex string
hex_constants = [f"{x:08x}" for x in constants_32bit]

# quick look at the first few hex constants
hex_constants[:10]


['428a2f98',
 '71374491',
 'b5c0fbcf',
 'e9b5dba5',
 '3956c25b',
 '59f111f1',
 '923f82a4',
 'ab1c5ed5',
 'd807aa98',
 '12835b01']

### Step 5 — Test the results against the Secure Hash Standard

The final check is to confirm that the values produced match the official SHA-256
constants from the Secure Hash Standard. A simple comparison is enough here to
show that all 64 generated constants line up with the expected values.


In [11]:
 # First 64 SHA-256 constants from the Secure Hash Standard (FIPS 180-4)
official_constants = [
    "428a2f98","71374491","b5c0fbcf","e9b5dba5","3956c25b","59f111f1","923f82a4","ab1c5ed5",
    "d807aa98","12835b01","243185be","550c7dc3","72be5d74","80deb1fe","9bdc06a7","c19bf174",
    "e49b69c1","efbe4786","0fc19dc6","240ca1cc","2de92c6f","4a7484aa","5cb0a9dc","76f988da",
    "983e5152","a831c66d","b00327c8","bf597fc7","c6e00bf3","d5a79147","06ca6351","14292967",
    "27b70a85","2e1b2138","4d2c6dfc","53380d13","650a7354","766a0abb","81c2c92e","92722c85",
    "a2bfe8a1","a81a664b","c24b8b70","c76c51a3","d192e819","d6990624","f40e3585","106aa070",
    "19a4c116","1e376c08","2748774c","34b0bcb5","391c0cb3","4ed8aa4a","5b9cca4f","682e6ff3",
    "748f82ee","78a5636f","84c87814","8cc70208","90befffa","a4506ceb","bef9a3f7","c67178f2"
]

# Compare our generated constants (hex_constants) with the official list
matches = [hex_constants[i] == official_constants[i] for i in range(64)]

# Show whether all matched
all(matches), matches.count(True)

(True, 64)